# GOES-DL Usage Examples (GridSat-B1)

**GOES-DL** is an open source tool to download GOES (Geostationary Operational
Environmental Satellite) 2nd generation (GOES-8 to GOES-15) data from NOAA's
[NCEI archive](https://www.ncdc.noaa.gov/gridsat/conusgoes-index.php), 4th Generation (GOES-16 to GOES-18) data from [NOAA's archive on AWS](https://aws.amazon.com/marketplace/pp/prodview-ngejrbcumyjtu), and Gridded Satellite B1 (GridSat-B1) Climate Data Record, Version 2, from [NOAA's AWS](https://aws.amazon.com/marketplace/pp/prodview-jir2e4n66gfyo) and [NCEI archives](https://www.ncdc.noaa.gov/gridsat/gridsat-index.php) using Python.

## Download data from GridSat-B1 Climate Data Record (Version 2) dataset

GridSat-B1 Climate Data Record are accessible via the NOAA archive hosted on AWS and through NOAA’s National Centers for Environmental Information archive.

This tutorial covers the following topics:

1. Using GOES-DL:
   1. Importing the `GOES_DL` package
   2. Importing the product locator class
   3. Importing the datasource class
   4. Importing the downloader class
2. Initialising the downloader
3. Downloading the data
   1. Getting the data file for a single date
   2. Getting data files for a range of dates

### 1. Using GOES-DL

#### 1.1. Importing the `goesdl` package

You can access the GOES-DL library's functionalities by simply importing the `goesdl` package:

In [1]:
import goesdl

or by specifically importing the required modules.

The library provides three main classes with which data downloads are configured. The first main class is the product locator. The product locator provides the necessary information to find the data files in the dataset repository. This is the only step that is specific to each dataset being downloaded. Product locators are defined in the `goesdl.dataset` subpackage.

#### 1.2. Importing the product locator

There is only one product locator for the GridSat-B1 dataset, it is defined in the `locator_b1` module under the `gridsat` subpackage:

In [2]:
from goesdl.dataset.gridsat import GridSatProductLocatorB1 as ProductLocator

The `GridSatProductLocatorB1` constructor has a single parameter, `versions`, initialized by default to the value of the only version available from the datasource; so you don't need to change its value. The parameter can take as value a string or a list of strings of the desired versions of the data files.

#### 1.3. Importing the datasource class

The second main class represents the datasource. `goesdl` provides two implementations focused on obtaining information from data repositories hosted on NCEI's HTTP servers or stored by NOAA in the Amazon Web Services cloud (AWS). The datasource provides the necessary functionality to access the data files from the repository, it abstracts the complexity of accessing data from different sources. Datasource classes are defined in the `goesdl.datasource` subpackage and are common for all datasets, just import the appropriate class for your use case.

The GridSat-B1 dataset is hosted on both, NCEI's HTTP servers and AWS cloud storage. Let's use the AWS datasource class:

In [3]:
from goesdl.datasource import DatasourceAWS as Datasource

The datasource objects, `DatasourceAWS` and `DatasourceHTTP`, are initialised with the previously created locator object. It can optionally take as a second argument a `DatasourceCache` object, which can speed up the download if you plan to download multiple data packages from the same datasource folder. By default, downloads are not stored in any cache.

#### 1.4. Importing the downloader class

The third and final main class is the `Downloader` class. The downloader is the main interface for downloading data and is common to all use cases. It is initialized with the datasource and locator objects, as well as the date format to be used in the download process, optionally. In most situations you will want to use a more compressed format; data files are not typically stored every minute or second. By default, it uses the ISO 8601 date format: `YYYY-mm-DDTHH:MM:SSz`, e.g. `1970-04-20T00:00:00-04:00` or `2024-08-23T00:00:00Z`.

In [4]:
from goesdl.downloader import Downloader

### 2. Initialising the downloader

Here we first instantiate a product locator object and with it we initialize the datasource object. Then, with both of them, we create the downloader object that we will use to download the data files:

In [5]:
# Initialize the downloader for GridSat-B1 (use the current directory as the local repository)

locator = ProductLocator()

datasource = Datasource(locator)

downloader = Downloader(
    datasource=datasource,
    locator=locator,
    repository="./1984-08-23",
    date_format="%Y-%m-%dT%H:%M%z",
)

### 3. Downloading the data

#### 3.1. Getting the data file for a single date

To download a data file, you just invoke the `download_files()` method on the downloader object providing a single `start` date. `download_files()` will return a list with the paths for the retrieved files, if any, relative to the path of the local repository, otherwise, an empty list will be returned.

In [6]:
# Set your desired date...
files1 = downloader.download_files(start="1984-08-23T00:00Z")

Retrieving available file list
1/1 1984/GRIDSAT-B1.1984.08.23.00.v02r01.nc
    ... downloaded succesfully


#### 3.2. Getting data files for a range of dates

To download data files for a range of dates, invoke the `download_files()` method on the downloader object providing a `start` date and an `end` date, both are included in the download range.

In [7]:
# ...or your desired date range
files2 = downloader.download_files(start="1984-08-23T00:00Z", end="1984-08-23T03:00Z")

Retrieving available file list
1/2 1984/GRIDSAT-B1.1984.08.23.00.v02r01.nc
    ... already downloaded
2/2 1984/GRIDSAT-B1.1984.08.23.03.v02r01.nc
    ... downloaded succesfully


#### 3.3. Custom filtering

To download only a subset of files within the requested range, invoke the `list_files()` method on the downloader object providing a `start` date and an `end` date, then operate on the returned list before calling to `get_files()` to retrieve the required datasets.

In [8]:
# Get the list of available files
files3 = downloader.list_files(start="1984-08-23T00:00Z", end="1984-08-24T04:00Z")

print("Available files")
print("---------------")
for file in files3:
    print(file)
print()

# ...just a dumb slicing
filtered = files3[:3]

# ...and get the files
downloader.get_files(file_paths=filtered)

Retrieving available file list
Available files
---------------
1984/GRIDSAT-B1.1984.08.23.00.v02r01.nc
1984/GRIDSAT-B1.1984.08.23.03.v02r01.nc
1984/GRIDSAT-B1.1984.08.23.06.v02r01.nc
1984/GRIDSAT-B1.1984.08.23.09.v02r01.nc
1984/GRIDSAT-B1.1984.08.23.12.v02r01.nc
1984/GRIDSAT-B1.1984.08.23.15.v02r01.nc
1984/GRIDSAT-B1.1984.08.23.18.v02r01.nc
1984/GRIDSAT-B1.1984.08.23.21.v02r01.nc
1984/GRIDSAT-B1.1984.08.24.00.v02r01.nc
1984/GRIDSAT-B1.1984.08.24.03.v02r01.nc

1/3 1984/GRIDSAT-B1.1984.08.23.00.v02r01.nc
    ... already downloaded
2/3 1984/GRIDSAT-B1.1984.08.23.03.v02r01.nc
    ... already downloaded
3/3 1984/GRIDSAT-B1.1984.08.23.06.v02r01.nc
    ... downloaded succesfully


filtered**NOTE**: Any error will raise an exception object, refer to the documentation for detailed information on the API and its behaviour.